In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from datetime import date, datetime, timedelta
import sqlalchemy
import pymysql
import openpyxl
import glob
from ast import literal_eval
from collections import Counter

In [7]:
load_dotenv()

hyperlink_path = 'json_completed_hyperlinks_update.csv'
parent_dir = os.getenv('dir')
data_dir = os.path.join(parent_dir,'data_update')

df = pd.read_csv(hyperlink_path, header=None)
df.head()
df.columns = ['ParseID','Hyperlink']
hyperlinks = df['Hyperlink'].tolist()

def foldername(hyperlink):
    hyperlink = hyperlink.split('/')[-1]
    return hyperlink[0:-8]
def providers_path(folder):
    return os.path.join(data_dir,folder,folder+'_providers.csv')

folder_names= [foldername(hyperlink) for hyperlink in hyperlinks]
provider_files = [providers_path(folder_name) for folder_name in folder_names]

ccn = pd.read_parquet('tin_to_ccn.parquet')

In [13]:
ccn = ccn.astype({'tin':'int64'})
ccn.dtypes

tin                     int64
npi_provider_groups    object
ccn                    object
dtype: object

In [18]:
ccn.dropna(subset=['ccn'],inplace=True)
ccn.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3895 entries, 96 to 46853
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   tin                  3895 non-null   int64 
 1   npi_provider_groups  3895 non-null   object
 2   ccn                  3895 non-null   object
dtypes: int64(1), object(2)
memory usage: 121.7+ KB


In [23]:
df = pd.read_csv(provider_files[2], usecols=['provider_reference','tin','npi_provider_groups'], converters={'npi_provider_groups': literal_eval})
provider_files[0]

provider_reference      int64
tin                     int64
npi_provider_groups    object
dtype: object

In [24]:
merged = df.merge(ccn[['tin','ccn']], how='inner', on=['tin'])
merged.head()

,provider_reference,tin,npi_provider_groups,ccn
0,0,1356638811,[1356638811],297112
1,1,1285698381,[1285698381],030101
2,1106,1285698381,[1285698381],030101
3,11,1508899253,[1508899253],290007
4,34,1932106853,[1932106853],030055


In [ ]:
def merged_prov_ccn(file_path):
    df = pd.read_csv(file_path, usecols=['provider_reference','tin','npi_provider_groups'], converters={'npi_provider_groups': literal_eval})
    df = df.astype({'tin':'int64'})
    merged = df.merge(ccn[['tin','ccn']], how='inner', on=['tin'])
    return merged

def save_file(df, path):
    df.to_pickle(path)
    print('File saved to pickle: \n' + path)
    

In [26]:
for i, provider in enumerate(provider_files):
    merged = merged_prov_ccn(provider)
    save_file(merged, os.path.join(data_dir,folder_names[i],folder_names[i]+'_ccn.parquet') )

'2023-01-01_ALL-SAVERS-INSURANCE-COMPANY_Insurer_PPO---NDC_PPO-NDC_in-network-rates'